## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
import pandas as pd
import numpy as np
%tensorflow_version 2.x
import tensorflow as tf

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd

prices = pd.read_csv('/content/drive/My Drive/prices.csv', nrows=1000)

### Check all columns in the dataset

In [5]:
prices.shape

(1000, 7)

In [6]:
iris.shape

(150, 6)

In [7]:
prices.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

In [8]:
iris.columns

Index(['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm',
       'Species'],
      dtype='object')

### Drop columns `date` and  `symbol`

In [9]:
prices.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [0]:
prices.drop(['date', 'symbol'], axis = 1, inplace=True)

In [11]:
prices.columns

Index(['open', 'close', 'low', 'high', 'volume'], dtype='object')

### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [12]:
# Already considered 1000 records while reading the dataset in the start
prices['volume'] = prices['volume'] / 1000000
prices.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

X = prices.drop('volume', axis=1)
y = prices['volume']
train_x,test_x,train_y,test_y=train_test_split(X,y,test_size=.30,random_state=2)

#### Convert Training and Test Data to numpy float32 arrays


In [14]:
train_x.dtypes

open     float64
close    float64
low      float64
high     float64
dtype: object

In [0]:
train_x =np.array(train_x).astype('float32')
test_x = np.array(test_x).astype('float32')
train_y =np.array(train_y).astype('float32')
test_y = np.array(test_y).astype('float32')
actual_prices=tf.cast(y,tf.float32)

In [16]:
train_x.dtype

dtype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

transformer = Normalizer()
train_x = transformer.fit_transform(train_x)
test_x = transformer.transform(test_x)

In [18]:
train_y.shape

(700,)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
#We are initializing weights and Bias of (Input-Hidden Layer)
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

In [20]:
print(w)
print(b)

tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]], shape=(4, 1), dtype=float32)
tf.Tensor([0.], shape=(1,), dtype=float32)


2.Define a function to calculate prediction

In [0]:
@tf.function
def prediction(train_x, w, b):
    y_pred = tf.add(tf.matmul(train_x, w), b)
    return y_pred

3.Loss (Cost) Function [Mean square error]

In [0]:
@tf.function
def loss(y_actual, y_predicted):
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
@tf.function
def train(x, y_actual, w, b, learning_rate=0.01):
    
    # Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        t.watch([w,b])
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    # Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    # Update Weights and Bias
    w = w - learning_rate * dw
    b = b - learning_rate * db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [48]:
for i in range(100):    
    w, b = train(train_x, train_y, w, b)
    print('Current Training Loss on iteration', i, loss(train_y, prediction(train_x, w, b)).numpy())

Current Training Loss on iteration 0 248.06491
Current Training Loss on iteration 1 245.7581
Current Training Loss on iteration 2 243.63223
Current Training Loss on iteration 3 241.6734
Current Training Loss on iteration 4 239.86803
Current Training Loss on iteration 5 238.20386
Current Training Loss on iteration 6 236.67052
Current Training Loss on iteration 7 235.25743
Current Training Loss on iteration 8 233.95468
Current Training Loss on iteration 9 232.7543
Current Training Loss on iteration 10 231.64807
Current Training Loss on iteration 11 230.62857
Current Training Loss on iteration 12 229.68898
Current Training Loss on iteration 13 228.82315
Current Training Loss on iteration 14 228.02501
Current Training Loss on iteration 15 227.28949
Current Training Loss on iteration 16 226.61156
Current Training Loss on iteration 17 225.98685
Current Training Loss on iteration 18 225.41127
Current Training Loss on iteration 19 224.88052
Current Training Loss on iteration 20 224.39154
Curre

### Get the shapes and values of W and b

In [53]:
w.numpy()

array([[1.3868148],
       [1.391616 ],
       [1.3725047],
       [1.4037081]], dtype=float32)

In [54]:
b.numpy()

array([2.77748], dtype=float32)

### Model Prediction on 1st Examples in Test Dataset

In [57]:
test_data=train_x[3:4,:]
test_data

array([[0.50063473, 0.5007126 , 0.49557468, 0.50304806]], dtype=float32)

In [58]:
pred=frwrd_pass(test_data,w,b)
pred

<tf.Tensor: id=5079, shape=(1, 1), dtype=float32, numpy=array([[5.554878]], dtype=float32)>

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris = pd.read_csv('/content/drive/My Drive/Iris.csv')

In [0]:
tf.random.set_seed(42)

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [98]:
iris.shape

(150, 6)

In [99]:
iris.dtypes

Id                 int64
SepalLengthCm    float64
SepalWidthCm     float64
PetalLengthCm    float64
PetalWidthCm     float64
Species            int64
dtype: object

In [112]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'species'. 
iris['Species']= label_encoder.fit_transform(iris['Species']) 
  
iris['Species'].unique() 

array([0, 1, 2])

### Splitting the data into feature set and target set

In [113]:
X = iris.drop(['Species','Id'] , axis=1)
y = tf.keras.utils.to_categorical(iris.Species, num_classes=3)
y.shape

(150, 3)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
# Initialize Sequential model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(4, input_dim=4, kernel_initializer='normal', activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [0]:
# Create optimizer with non-default learning rate
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)

# Compile the model
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

### Model Training 

### Model Prediction

In [121]:
trainX,testX,trainY,testY=train_test_split(X,y,test_size=.30,random_state=2)

model.fit(trainX, trainY, validation_data=(testX, testY), epochs=40)

Train on 105 samples, validate on 45 samples
Epoch 1/40
105/105 [==============================] - 0s 165us/sample - loss: 0.4365 - accuracy: 0.8190 - val_loss: 0.4271 - val_accuracy: 0.8222
Epoch 2/40
105/105 [==============================] - 0s 120us/sample - loss: 0.4360 - accuracy: 0.8381 - val_loss: 0.4225 - val_accuracy: 0.9778
Epoch 3/40
105/105 [==============================] - 0s 126us/sample - loss: 0.4389 - accuracy: 0.9048 - val_loss: 0.4293 - val_accuracy: 0.7111
Epoch 4/40
105/105 [==============================] - 0s 115us/sample - loss: 0.4341 - accuracy: 0.8286 - val_loss: 0.4221 - val_accuracy: 0.8444
Epoch 5/40
105/105 [==============================] - 0s 111us/sample - loss: 0.4342 - accuracy: 0.8667 - val_loss: 0.4197 - val_accuracy: 0.8667
Epoch 6/40
105/105 [==============================] - 0s 120us/sample - loss: 0.4308 - accuracy: 0.8762 - val_loss: 0.4186 - val_accuracy: 0.8667
Epoch 7/40
105/105 [==============================] - 0s 130us/sample - loss: 0

In [122]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_17 (Dense)             (None, 3)                 15        
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________


In [124]:
model.evaluate(testX, testY)

45/45 [==============================] - 0s 102us/sample - loss: 0.3842 - accuracy: 0.9778


[0.38424985739919876, 0.9777778]

### Save the Model

In [0]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?